# 1. 환경 셋업

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import boto3
# import json

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)

python path: /Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery is added
sys.path:  ['/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/joohyery/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages', '/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery']


In [3]:
# 필요한 변수들 정의

region = "us-west-2"
nova_pro_model_id = "us.amazon.nova-pro-v1:0"
#claude3_7_model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

client = boto3.client(service_name="bedrock-runtime", region_name=region)


# 2. 샘플 포트폴리오 경로 로딩 하기

In [4]:
current_folder = os.getcwd()
print(current_folder)

/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer


In [5]:
portfolio_path = os.path.join(os.getcwd(), 'portfolio')

# 해당 폴더 내 파일/디렉토리 리스트 출력
if os.path.exists(portfolio_path):
    for filename in os.listdir(portfolio_path):
        print(filename)
else:
    print("'portfolio' 폴더가 현재 위치에 없습니다.")

target = "None"

skincare.png
tech.png


# 3. 포트폴리오 로딩 및 특징 파악

In [6]:
analysis_system_prompt = f"""당신은 웹 디자인 및 개발 전문가로서, 웹사이트 제작 포트폴리오 샘플 이미지를 분석하여 특징과 특성을 파악하는 역할을 수행합니다. 당신의 분석 결과는 웹사이트 제작을 고려하는 브랜드 사업자들에게 유용한 인사이트를 제공할 것입니다.

## 입력 형식
<target_image>
{str(target)}
</target_image>

## 주요 역할
웹사이트 제작 관련 포트폴리오 이미지를 체계적으로 분석하여 디자인, 기술적 구현, 사용자 경험 등의 관점에서 특징을 도출하고 설명하세요.

## 분석 항목

### 1. 디자인 분석
- **레이아웃 구조**: 헤더, 네비게이션, 메인 콘텐츠, 사이드바, 푸터 등의 배치와 구성을 세밀히 분석합니다. 각 섹션의 크기, 비율, 위치가 사용자 시선 흐름에 미치는 영향을 평가하고, 정보 계층구조가 명확하게 구성되어 있는지 확인합니다.
- **색상 스키마**: 주요 색상, 보조 색상, 배색 조합의 특성과 브랜드 일관성을 분석합니다. 색상의 심리적 효과, 대비도, 접근성을 고려한 색상 선택의 적절성을 평가하며, 전체적인 분위기와 브랜드 아이덴티티와의 일치성을 검토합니다.
- **타이포그래피**: 폰트 종류, 크기, 계층 구조, 가독성을 종합적으로 분석합니다. 제목, 부제목, 본문의 크기 비율이 적절한지, 글꼴의 스타일이 웹사이트의 목적과 브랜드 이미지에 부합하는지 평가합니다.
- **시각적 요소**: 아이콘, 이미지, 그래픽 요소의 스타일과 활용도를 분석합니다. 일러스트레이션, 사진, 아이콘의 통일성과 품질, 그리고 콘텐츠와의 조화를 평가하며, 시각적 노이즈가 없는지 확인합니다.
- **화이트스페이스**: 여백 활용과 전체적인 균형감을 분석합니다. 콘텐츠 간의 간격, 여백의 일관성, 시각적 휴식공간의 적절한 배치를 평가하여 가독성과 미적 균형을 검토합니다.

### 2. 기능 및 구조 분석
- **네비게이션**: 메뉴 구조, 사용자 경로, 접근성을 종합적으로 평가합니다. 메인 메뉴, 서브 메뉴, 브레드크럼, 검색 기능의 배치와 사용성을 분석하고, 사용자가 원하는 정보에 쉽게 도달할 수 있는 경로가 제공되는지 확인합니다.
- **콘텐츠 조직**: 정보 구조화, 콘텐츠 우선순위, 페이지 흐름을 분석합니다. 중요한 정보가 적절한 위치에 배치되어 있는지, 사용자의 관심을 끌고 행동을 유도하는 구조로 설계되어 있는지 평가합니다.
- **인터랙션 요소**: 버튼, 링크, 폼 요소의 디자인과 배치를 분석합니다. CTA(Call to Action) 버튼의 시각적 강조, 클릭 가능한 요소의 명확성, 사용자 피드백 제공 여부를 평가합니다.
- **반응형 디자인**: 다양한 화면 크기에 대한 적응성을 추론하여 분석합니다. 모바일, 태블릿, 데스크톱 환경에서의 레이아웃 변화와 사용성을 고려한 설계가 되어 있는지 평가합니다.

### 3. 사용자 경험(UX) 분석
- **가독성**: 텍스트와 콘텐츠의 읽기 용이성을 분석합니다. 폰트 크기, 줄 간격, 단락 구분, 배경과 텍스트의 대비 등이 사용자의 읽기 경험에 미치는 영향을 평가합니다.
- **직관성**: 사용자가 쉽게 이해할 수 있는 인터페이스 구성을 분석합니다. 아이콘의 의미 전달력, 버튼과 링크의 명확성, 전체적인 정보 구조의 논리성을 평가합니다.
- **접근성**: 장애인 접근성을 고려한 디자인 요소를 분석합니다. 색상만에 의존하지 않는 정보 전달, 충분한 대비도, 키보드 탐색 가능성 등을 평가합니다.
- **사용자 목표**: 웹사이트의 주요 목적과 사용자 행동 유도 효과를 분석합니다. 비즈니스 목표와 사용자 니즈의 균형, 전환율 향상을 위한 설계 요소를 평가합니다.

### 4. 기술적 특징 추론
- **개발 기술**: 사용된 것으로 추정되는 프론트엔드 기술이나 프레임워크
- **성능 최적화**: 이미지 최적화, 로딩 속도 고려사항
- **브라우저 호환성**: 크로스 브라우징 고려사항

### 5. 업계 및 목적 분석
- **타겟 업종**: 포트폴리오가 대상으로 하는 산업이나 분야를 분석합니다. 업종의 특성과 요구사항에 맞는 디자인과 기능이 구현되어 있는지 평가합니다.
- **비즈니스 목표**: 웹사이트의 주요 비즈니스 목적을 분석합니다. 브랜드 인식 증진, 리드 생성, 판매 촉진, 고객 서비스 등의 목표에 적합한 구조인지 평가합니다.
- **타겟 사용자**: 주요 사용자층과 그들의 니즈를 분석합니다. 연령대, 기술 사용 능력, 관심사, 행동 패턴에 맞는 사용자 경험을 제공하는지 평가합니다.

## 출력 형식
분석 결과를 다음과 같이 구조화하여 제시하세요:

1. **전체적인 인상**: 첫 인상과 전반적인 특징을 상세히 기술
2. **강점**: 돋보이는 디자인이나 기능적 요소를 구체적으로 설명
3. **개선 가능한 영역**: 잠재적 개선점이나 고려사항을 건설적으로 제시
4. **기술적 특성**: 사용된 기술이나 구현 방식에 대한 추론
5. **업계 적합성**: 해당 업종이나 목적에 대한 적합성을 종합적으로 평가

## 주의사항
- target_image 태그 안에 제공된 포트폴리오 이미지에서 직접 관찰할 수 있는 요소에 기반하여 분석하세요
- 추론이나 가정이 필요한 부분은 명확히 구분하여 표시하세요
- 객관적이고 전문적인 관점에서 분석하되, 건설적인 피드백을 제공하세요
- 한국의 웹 디자인 트렌드, 사용자 습관, 문화적 특성을 고려하여 분석하세요
- 이미지가 명확하지 않거나 부분적으로만 보이는 경우, 관찰 가능한 범위 내에서 분석하고 한계를 명시하세요
"""

In [ ]:
import base64
from PIL import Image
import io
from botocore.exceptions import ClientError

# Bedrock Converse API의 이미지 크기 제한은 3.75MB라 이미지를 3.75MB 이하로 압축하는 코드가 필요
def compress_image(image_path, max_size_mb=5):
    """
    이미지를 지정된 크기 이하로 압축하는 함수
    
    Args:
        image_path (str): 원본 이미지 경로
        max_size_mb (int): 최대 크기 (MB)
    
    Returns:
        bytes: 압축된 이미지 바이트 데이터
        str: 이미지 포맷
    """
    max_size_bytes = max_size_mb * 1024 * 1024  # MB를 바이트로 변환
    
    # 이미지 열기
    with Image.open(image_path) as img:
        # 원본 포맷 확인
        original_format = img.format.lower() if img.format else 'png'
        
        # RGB로 변환 (JPEG 저장을 위해)
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
            
        # 초기 품질 설정
        quality = 95
        
        while quality > 10:  # 최소 품질 10까지 시도
            # 이미지를 메모리 버퍼에 저장
            buffer = io.BytesIO()
            
            # JPEG로 압축 (가장 효율적)
            img.save(buffer, format='JPEG', quality=quality, optimize=True)
            buffer.seek(0)
            
            # 압축된 크기 확인
            compressed_size = len(buffer.getvalue())
            
            if compressed_size <= max_size_bytes:
                print(f"이미지 압축 완료: {compressed_size:,} bytes (품질: {quality})")
                return buffer.getvalue(), 'jpeg'
            
            # 품질을 10씩 감소
            quality -= 10
        
        # 품질을 낮춰도 안 되면 크기를 줄임
        print("품질 조정으로 압축 실패. 이미지 크기를 조정합니다...")
        
        # 이미지 크기 단계적 축소
        scale_factors = [0.8, 0.6, 0.4, 0.2]
        
        for scale in scale_factors:
            # 이미지 크기 축소
            new_width = int(img.width * scale)
            new_height = int(img.height * scale)
            resized_img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            
            # 압축된 이미지 저장
            buffer = io.BytesIO()
            resized_img.save(buffer, format='JPEG', quality=85, optimize=True)
            buffer.seek(0)
            
            compressed_size = len(buffer.getvalue())
            
            if compressed_size <= max_size_bytes:
                print(f"이미지 리사이징 및 압축 완료: {compressed_size:,} bytes (크기: {new_width}x{new_height})")
                return buffer.getvalue(), 'jpeg'
        
        # 최후의 수단: 매우 작은 크기로 축소
        resized_img = img.resize((800, 600), Image.Resampling.LANCZOS)
        buffer = io.BytesIO()
        resized_img.save(buffer, format='JPEG', quality=70, optimize=True)
        buffer.seek(0)
        
        print(f"최종 압축 완료: {len(buffer.getvalue()):,} bytes (800x600, 품질: 70)")
        return buffer.getvalue(), 'jpeg'



In [ ]:
def analyze_portfolio_image(target_path):
    try:
        # 이미지 압축
        print(f"원본 이미지 크기 확인 중...")
        original_size = os.path.getsize(target_path)
        print(f"원본 크기: {original_size:,} bytes ({original_size/1024/1024:.1f} MB)")
        
        if original_size > 3.75 * 1024 * 1024:  # 5MB 초과
            print("이미지가 3.75MB를 초과합니다. 압축을 시작합니다...")
            image_bytes, image_format = compress_image(target_path)
        else:
            # 3.75MB 이하면 원본 사용
            with open(target_path, "rb") as image_file:
                image_bytes = image_file.read()
            
            # 원본 포맷 확인
            file_extension = os.path.splitext(target_path)[1].lower()
            if file_extension == '.png':
                image_format = 'png'
            elif file_extension in ['.jpg', '.jpeg']:
                image_format = 'jpeg'
            else:
                image_format = 'png'

        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "text": f"<target_image>\n{target_path}\n</target_image>\n\n제공된 웹사이트 포트폴리오 이미지를 분석해주세요. (원본 크기가 커서 압축했을 수 있습니다)"
                    },
                    {
                        "image": {
                            "format": image_format,
                            "source": {
                                "bytes": image_bytes
                            }
                        }
                    }
                ]
            }
        ]

        print("Bedrock API 호출 중...")
        response = client.converse(
            modelId=nova_pro_model_id,
            messages=messages,
            system=[
                {
                    "text": analysis_system_prompt
                },
                {
                    "cachePoint": {
                        "type": "default"
                    }
                }
            ],
            inferenceConfig={
                "maxTokens": 4000,
                "temperature": 0.1,
                "topP": 0.9
            }
        )

        return response
    
    except FileNotFoundError:
        print(f"이미지 파일을 찾을 수 없습니다: {target_path}")
        return None
    except ClientError as e:
        print(f"Bedrock API 오류: {e}")
        return None
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")
        return None

In [9]:
def main():
    
    current_folder = os.getcwd()
    print(current_folder)

    portfolio_path = os.path.join(os.getcwd(), 'portfolio')
    print(portfolio_path)

    if os.path.exists(portfolio_path):
        for filename in os.listdir(portfolio_path):
            print(filename)
    else:
        print("'portfolio' 폴더가 현재 위치에 없습니다.")

    if os.path.exists(portfolio_path):
        for filename in os.listdir(portfolio_path):
            if filename == target:
                target_path = f"portfolio/{filename}"
                break
        
        if target:
            print(f"파일을 찾았습니다: {target}")
        else:
            print("파일을 찾을 수 없습니다.")
    else:
        print("portfolio 폴더가 존재하지 않아 파일을 찾을 수 없습니다.")

    # 이미지 분석 실행
    if target and os.path.exists(target_path):
        print(f"\n=== 포트폴리오 이미지 분석 시작 ===")
        print(f"분석 대상: {target}")
        print(f"사용 모델: {nova_pro_model_id}")
        
        result = analyze_portfolio_image(target_path)
        
        if result:
            analysis_text = result['output']['message']['content'][0]['text']
            print(f"\n=== 분석 결과 ===")
            print(analysis_text)

            usage = result['usage']
            print(f"\n=== 사용량 정보 ===")
            
            # 캐시 관련 토큰 정보 추출
            cache_write = usage.get('cacheWriteInputTokens', 0)
            cache_read = usage.get('cacheReadInputTokens', 0)
            
            # 캐시 상태 확인 및 출력
            if cache_write > 0 and cache_read == 0:
                print("첫 번째 실행: 캐시가 새로 생성되었습니다.")
                print(f"   - System prompt가 캐시에 저장됨: {cache_write} 토큰")
            elif cache_read > 0:
                print("캐시 사용: 이전에 생성된 캐시를 사용했습니다.")
                print(f"   - System prompt를 캐시에서 읽음: {cache_read} 토큰")
            else:
                print("캐시 정보 없음: 캐싱이 활성화되지 않았을 수 있습니다.")
            
            # 전체 토큰 사용량 정보
            print(f"\n=== 전체 토큰 사용량 ===")
            print(f"Input Token: {usage['inputTokens']}")
            print(f"Output Token: {usage['outputTokens']}")
            print(f"Cache Write: {cache_write}")
            print(f"Cache Read: {cache_read}")

        else:
            print("이미지 분석에 실패했습니다.")
    else:
        print("분석할 이미지 파일이 없습니다.")

## 3-1. skincare 도메인 웹사이트 포트폴리오 분석

In [10]:
target = "skincare.png"
print("## This is target image")
print({target})

## This is target image
{'skincare.png'}


In [11]:
main()

/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer
/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer/portfolio
skincare.png
tech.png
파일을 찾았습니다: skincare.png

=== 포트폴리오 이미지 분석 시작 ===
분석 대상: skincare.png
사용 모델: us.amazon.nova-pro-v1:0
원본 이미지 크기 확인 중...
원본 크기: 10,816,637 bytes (10.3 MB)
이미지가 5MB를 초과합니다. 압축을 시작합니다...
이미지 압축 완료: 1,796,829 bytes (품질: 95)
Bedrock API 호출 중...

=== 분석 결과 ===
### 1. 전체적인 인상

이 웹사이트 포트폴리오는 화장품 브랜드의 웹사이트를 위한 디자인을 보여줍니다. 전반적으로 핑크와 베이지 톤을 기반으로 한 색상 스키마가 눈에 띄며, 여성스럽고 부드러운 이미지를 전달합니다. 레이아웃은 깔끔하고 단순하며, 각 섹션이 명확하게 구분되어 있습니다. 헤더에는 브랜드 로고와 주요 메뉴가 위치하고, 메인 콘텐츠 영역에는 제품 소개, 이벤트, 브랜드 스토리 등이 구성되어 있습니다. 푸터에는 브랜드 정보와 소셜 미디어 링크가 제공됩니다.

### 2. 강점

- **시각적 일관성**: 핑크와 베이지 톤을 기반으로 한 색상 스키마가 일관되게 적용되어 브랜드 이미지를 강화합니다.
- **제품 강조**: 제품 이미지가 크게 표시되어 사용자의 관심을 끌며, 제품 정보가 간결하게 제공됩니다.
- **이벤트 및 프로모션**: 이벤트 섹션이 별도로 구성되어 사용자에게 특별한 혜택을 제공하고자 하는 브랜드의 의도가 잘 반영되었습니다.
- **반응형 디자인**: 다양한 화면 크기에 적응하는 반응형 디자인이 적용되어 사용

## 3-2. tech 도메인 웹사이트 포트폴리오 분석

In [12]:
target = "tech.png"
print("## This is target image")
print({target})

## This is target image
{'tech.png'}


In [13]:
main()

/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer
/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer/portfolio
skincare.png
tech.png
파일을 찾았습니다: tech.png

=== 포트폴리오 이미지 분석 시작 ===
분석 대상: tech.png
사용 모델: us.amazon.nova-pro-v1:0
원본 이미지 크기 확인 중...
원본 크기: 9,460,007 bytes (9.0 MB)
이미지가 5MB를 초과합니다. 압축을 시작합니다...
이미지 압축 완료: 2,234,561 bytes (품질: 95)
Bedrock API 호출 중...

=== 분석 결과 ===
### 1. 전체적인 인상

이 웹사이트 포트폴리오는 기술 기업의 웹사이트를 보여주며, 청량감 있는 디자인과 현대적인 레이아웃을 특징으로 합니다. 푸른색과 회색을 주요 색상으로 사용하여 전문적이고 신뢰할 수 있는 이미지를 전달합니다. 헤더에는 로고와 주요 메뉴가 간결하게 배치되어 있으며, 메인 콘텐츠 영역은 다양한 서비스와 제품을 시각적으로 강조하고 있습니다. 전반적으로 깔끔하고 직관적인 사용자 경험을 제공하는 것으로 보입니다.

### 2. 강점

- **시각적 계층 구조**: 헤더, 메인 콘텐츠, 푸터의 구조가 명확하게 구분되어 있어 사용자가 쉽게 정보를 찾을 수 있습니다.
- **색상 스키마**: 푸른색과 회색의 조합이 전문적이고 신뢰할 수 있는 느낌을 줍니다.
- **반응형 디자인**: 다양한 화면 크기에 대한 적응성이 잘 구현되어 있어 모바일과 데스크톱 환경 모두에서 좋은 사용자 경험을 제공할 것으로 예상됩니다.
- **인터랙션 요소**: 버튼과 링크가 시각적으로 강조되어 사용자가 쉽게 클릭할 수 있습니다.

### 3. 개선 가능한 영역



# 4. 유저 취향 키워드 기반 포트폴리오 적합도 분석

In [14]:
# 모델에 포트폴리오 분석 결과와 함께 유저 취향 키워드를 주고, 포트폴리오가 유저 키워드와 얼마나 어울리는지(추천 가능성 퍼센트)와 이유 설명을 요청하는 함수

def analyze_portfolio_with_user_preferences(target_path, user_keywords, previous_analysis_output):
    try:
        additional_request = f"""
            유저 취향 키워드: {', '.join(user_keywords)}

            위의 포트폴리오 분석 결과를 바탕으로, 이 웹사이트 포트폴리오가 유저의 취향 키워드와 얼마나 잘 맞는지 다음 형식으로 평가해주세요:

            1. **적합도 점수**: 0-100% 사이의 퍼센트로 표현
            2. **적합도 이유**: 유저 키워드와 포트폴리오의 연관성을 구체적으로 설명
            3. **추천 여부**: 추천함/추천하지 않음과 그 근거

            각 유저 키워드별로 포트폴리오와의 연관성을 분석하고, 종합적인 적합도를 산출해주세요.
            """
        
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "text": f"<target_image>\n{target_path}\n</target_image>\n\n이전에 분석된 결과는 다음과 같습니다:\n\n{previous_analysis_output}\n\n{additional_request}"
                    }
                ]
            }
        ]

        response = client.converse(
            modelId=nova_pro_model_id,
            messages=messages,
            system=[{"text": analysis_system_prompt}],
            inferenceConfig={
                "maxTokens": 4000,
                "temperature": 0.3,
                "topP": 0.95
            }
        )

        return response

    except Exception as e:
        print(f"유저 키워드 기반 추가 분석 중 오류가 발생했습니다: {e}")
        return None

In [15]:
# 유저 취향 키워드 정의
user_keywords = [""]

In [16]:
def enhanced_main():
    
    current_folder = os.getcwd()
    print(current_folder)

    portfolio_path = os.path.join(os.getcwd(), 'portfolio')
    print(portfolio_path)

    if os.path.exists(portfolio_path):
        for filename in os.listdir(portfolio_path):
            print(filename)
    else:
        print("'portfolio' 폴더가 현재 위치에 없습니다.")

    if os.path.exists(portfolio_path):
        for filename in os.listdir(portfolio_path):
            if filename == target:
                target_path = f"portfolio/{filename}"
                break
        
        if target:
            print(f"파일을 찾았습니다: {target}")
        else:
            print("파일을 찾을 수 없습니다.")
    else:
        print("portfolio 폴더가 존재하지 않아 파일을 찾을 수 없습니다.")

    if target and os.path.exists(target_path):
        print(f"\n=== 포트폴리오 이미지 분석 시작 ===")
        print(f"분석 대상: {target}")
        print(f"사용 모델: {nova_pro_model_id}")
        
        # 기본 이미지 분석 실행
        result = analyze_portfolio_image(target_path)
        if result:
            previous_analysis_output = result['output']['message']['content'][0]['text']
            print(previous_analysis_output)
            
            # 기본 분석 사용량 정보
            usage1 = result['usage']
            cache_creation1 = usage1.get('cacheCreationInputTokens', 0)
            cache_read1 = usage1.get('cacheReadInputTokens', 0)
            
            print(f"\n=== 기본 분석 토큰 사용량 ===")
            print(f"Input Token: {usage1['inputTokens']}")
            print(f"Output Token: {usage1['outputTokens']}")
            print(f"Cache Write: {cache_creation1}")
            print(f"Cache Read: {cache_read1}")
            print(f"Total Token: {usage1['totalTokens']}")
            
            # 유저 취향 키워드 기반 적합도 분석
            print(f"\n=== 유저 취향 적합도 분석 시작 ===")
            print(f"유저 키워드: {', '.join(user_keywords)}")
            
            preference_result = analyze_portfolio_with_user_preferences(
                target, 
                user_keywords,
                previous_analysis_output
            )
            
            if preference_result:
                preference_text = preference_result['output']['message']['content'][0]['text']
                print(f"\n=== 유저 취향 적합도 분석 결과 ===")
                print(preference_text)
                
                # 적합도 분석 사용량 정보
                usage2 = preference_result['usage']
                cache_creation2 = usage2.get('cacheCreationInputTokens', 0)
                cache_read2 = usage2.get('cacheReadInputTokens', 0)
                
                print(f"\n=== 적합도 분석 토큰 사용량 ===")
                print(f"Input Token: {usage2['inputTokens']}")
                print(f"Output Token: {usage2['outputTokens']}")
                print(f"Cache Write: {cache_creation2}")
                print(f"Cache Read: {cache_read2}")
                print(f"Total Token: {usage2['totalTokens']}")
                
                # 전체 사용량 합계
                total_input = usage1['inputTokens'] + usage2['inputTokens']
                total_output = usage1['outputTokens'] + usage2['outputTokens']
                total_cache_write = cache_creation1 + cache_creation2
                total_cache_read = cache_read1 + cache_read2
                total_tokens = usage1['totalTokens'] + usage2['totalTokens']
                
                print(f"\n=== 전체 토큰 사용량 합계 ===")
                print(f"Total Input Token: {total_input}")
                print(f"Total Output Token: {total_output}")
                print(f"Total Cache Write: {total_cache_write}")
                print(f"Total Cache Read: {total_cache_read}")
                print(f"Grand Total Token: {total_tokens}")

            else:
                print("유저 취향 적합도 분석에 실패했습니다.")
                
        else:
            print("기본 이미지 분석에 실패했습니다.")
    else:
        print("분석할 이미지 파일이 없습니다.")


## 4-1. 웹사이트 포트폴리오 유저 추천

In [17]:
# 유저 취향 키워드 정의
user_keywords = ["뷰티", "MZ", "스킨케어 화장품", "메이크업"]

In [18]:
enhanced_main()

/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer
/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer/portfolio
skincare.png
tech.png
파일을 찾았습니다: tech.png

=== 포트폴리오 이미지 분석 시작 ===
분석 대상: tech.png
사용 모델: us.amazon.nova-pro-v1:0
원본 이미지 크기 확인 중...
원본 크기: 9,460,007 bytes (9.0 MB)
이미지가 5MB를 초과합니다. 압축을 시작합니다...
이미지 압축 완료: 2,234,561 bytes (품질: 95)
Bedrock API 호출 중...
### 1. 전체적인 인상

이 웹사이트 포트폴리오는 기술 기업의 웹사이트를 보여주며, 청량감 있는 디자인과 현대적인 레이아웃을 특징으로 합니다. 푸른색과 회색을 주로 사용하여 전문적이고 신뢰할 수 있는 이미지를 전달하고 있습니다. 메인 슬라이더와 다양한 섹션으로 구성되어 있으며, 각 섹션은 클릭 가능한 요소와 상호작용 기능을 통해 사용자 경험을 향상시키고 있습니다.

### 2. 강점

- **시각적 디자인**: 푸른색과 회색의 조합은 전문적이고 신뢰할 수 있는 느낌을 줍니다. 시각적으로 매력적인 슬라이더와 아이콘이 사용되어 사용자의 관심을 끌 수 있습니다.
- **명확한 레이아웃**: 헤더, 메인 슬라이더, 서비스 섹션, 제품 섹션, 솔루션 섹션, 파트너 섹션 등으로 구성되어 있어 사용자가 쉽게 필요한 정보를 찾을 수 있습니다.
- **상호작용 요소**: 각 섹션에는 클릭 가능한 요소와 상호작용 기능이 포함되어 있어 사용자 경험을 향상시킵니다.
- **반응형 디자인**: 다양한 화면 크기에 적응하는 반응형 디자인이 적용되어 있어 모바일, 태블릿, 데스크톱 등 다양한 기기에서

In [19]:
# 유저 취향 키워드 재정의
user_keywords = ["AI", "IT", "전문성"]

In [20]:
enhanced_main()

/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer
/Users/joohyery/Documents/Dev/amazon-bedrock-samples-joohyery/bedrock-web-portfolio-analyzer/portfolio
skincare.png
tech.png
파일을 찾았습니다: tech.png

=== 포트폴리오 이미지 분석 시작 ===
분석 대상: tech.png
사용 모델: us.amazon.nova-pro-v1:0
원본 이미지 크기 확인 중...
원본 크기: 9,460,007 bytes (9.0 MB)
이미지가 5MB를 초과합니다. 압축을 시작합니다...
이미지 압축 완료: 2,234,561 bytes (품질: 95)
Bedrock API 호출 중...
### 1. 전체적인 인상

이 웹사이트 포트폴리오는 기술 기업의 웹사이트를 보여주며, 청량감 있는 디자인과 현대적인 레이아웃을 특징으로 합니다. 푸른색과 회색을 주로 사용하여 전문적이고 신뢰할 수 있는 이미지를 전달하고 있습니다. 헤더에는 브랜드 로고와 주요 메뉴가 간결하게 배치되어 있으며, 메인 콘텐츠 영역은 다양한 서비스와 제품을 시각적으로 강조하고 있습니다.

### 2. 강점

- **시각적 디자인**: 푸른색과 회색의 조합이 전문적이고 신뢰할 수 있는 느낌을 줍니다. 배경 이미지와 함께 현대적인 분위기를 조성하고 있습니다.
- **명확한 레이아웃**: 헤더, 메인 콘텐츠, 푸터 구조가 명확하고 일관성 있게 구성되어 있습니다. 사용자가 쉽게 필요한 정보를 찾을 수 있도록 도와줍니다.
- **다양한 콘텐츠 표현**: 서비스, 제품, 솔루션 등 다양한 콘텐츠를 시각적으로 효과적으로 표현하고 있습니다. 이미지와 텍스트의 조합이 조화롭습니다.
- **반응형 디자인**: 다양한 화면 크기에 적응하는 반응형 디자인이 적용되어 있어 사용자 경험을 향상시킵니다.

### 3. 개선 